In [89]:
# import libraries
import polars as pl
import pandas as pd
import pyarrow.parquet as pq
import scipy.stats as sts
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
#import plotly.express as px
#import plotly.graph_objs as go
import dask
from dask.distributed import Client, LocalCluster
import dask.dataframe as dd
import dask.config
#import geopandas as gpd
# import osmnx as ox
import json
from datetime import datetime, timedelta, date
import os
import time
import sys
import requests
# import sidetable
import warnings
warnings.filterwarnings('ignore')

In [90]:
# Customize display options
dask.config.set({'dataframe.pprint.max_rows': 10,   # Set the maximum number of rows to display
                 'dataframe.pprint.max_columns': 10,  # Set the maximum number of columns to display
                 'dataframe.pprint.always_use_ascii': True,  # Use ASCII characters for formatting
                 'dataframe.pprint.unicode.ambiguous_as_wide': True,  # Use wide formatting for ambiguous unicode characters
                 'dataframe.pprint.float_format': '{:.2f}'})  # Set the formatting for floating-point values


In [91]:
sys.tracebacklimit = 0 # turn off long tracebacks on error messages
#sys.tracebacklimit = None # use to turn tracebacks back on if needed

In [92]:
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", None)

In [93]:
#extra columns. Keeping for reference

#drug treatment columns

#                 'therapeutic_bamlanivimab_number_of_residents_treated_from_stock_stored_at_this_facility',
#                 'therapeutic_bamlanivimab_number_of_residents_treated_from_stock_stored_at_another_facility',
#                 'therapeutic_casirivimab_plus_imdevimab_number_of_residents_treated_from_stock_stored_at_this_facility',
#                 'therapeutic_casirivimab_plus_imdevimab_number_of_residents_treated_from_stock_stored_at_another_facility',
#                 'therapeutic_bamlanivimab_plus_etesevimab_number_of_residents_treated_from_stock_stored_at_this_facility',
#                 'therapeutic_bamlanivimab_plus_etesevimab_number_of_residents_treated_from_stock_stored_at_another_facility',
#                 'therapeutic_sotrovimab_number_of_residents_treated_from_stock_stored_at_this_facility',
#                 'therapeutic_sotrovimab_number_of_residents_treated_from_stock_stored_at_another_facility',
#                 'therapeutic_evusheld_number_of_residents_treated_from_stock_stored_at_this_facility',
#                 'therapeutic_evusheld_number_of_residents_treated_from_stock_stored_at_another_facility',
#                 'therapeutic_molnupiravir_number_ of_residents_treated_from_stock_stored_at_this_facility',
#                 'therapeutic_molnupiravir_number_ of_residents_treated_from_stock_stored_at_another_facility',
#                 'therapeutic_paxlovid_number_ of_residents_treated_from_stock_stored_at_this_facility',
#                 'therapeutic_paxlovid_number_ of_residents_treated_from_stock_stored_at_another_facility',
#                 'therapeutic_bebtelovimab_number_of_residents_treated from_stock_stored_at_this_facility',
#                'therapeutic_bebtelovimab_number_of_residents_treated from_stock_stored_at_another_facility',

# covid19 vaccine columns
#                'recent_percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time',
#                 'percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time',
#                 'recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time',
#                 'percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time',
#                 'number_of_residents_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time',
#                 'number_of_healthcare_personnel_with_a_completed_vaccination_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time',
#                 'number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time',
#                  'number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time'
# flu columns
#                 'number_of_residents_with_new_influenza',
#                 'number_of_staff_and_or_personnel_with_new_influenza',

## Key Questions, Info, and Sources

### Who reports this data and how often?


### What facilities does this data include? What does it exclude? And why?


### What date range do I want to look at? When is the earliest that CMS has reliable vax data for c19? 
The vaccination data starts to be consistently reported at majority of facilities is June 2021. I care about tracking vaccination at anytime metrics from that date through the year, so start on week ending Sunday 7-4-2021 and stop on week ending Sunday, 7-3-2022.

### When did the vaccines and boosters come out?

"
March 29, 2022: FDA authorized the second booster dose of either the Pfizer-BioNTech or the Moderna COVID-19 vaccines for adults ages 50 years and older and certain immunocompromised individuals.

January 31, 2022
FDA approved the second COVID-19 vaccine, Spikevax (COVID-19 Vaccine, mRNA), which was previously known as Moderna COVID-19 Vaccine; the approved vaccine will be marketed as Spikevax for the prevention of COVID-19 in individuals 18 years of age and older.

November 21, 2021
HHS Secretary issued a directive to expand eligibility for all adults ages 18 and older - PDF to receive a booster dose of COVID-19 vaccines.

"


https://www.hhs.gov/coronavirus/covid-19-vaccines/index.html#:~:text=During%20the%20COVID%2D19%20pandemic,began%20on%20December%2014%2C%202020.



### What fields do we actualy care about?  From CMS site: https://data.cms.gov/covid-19/covid-19-nursing-home-data:

* Recent Percentage of Current Residents who Received a Completed COVID-19 Vaccination at Any Time
* Recent Percentage of Current Residents Up to Date with COVID-19 Vaccines
* Recent Percentage of Current Healthcare Personnel who Received a Completed COVID-19 Vaccination at Any Time
* Recent Percentage of Current Healthcare Personnel Up to Date with COVID-19 Vaccines

### Rounding back to the data dictionary for clarity on how these are calculated:
https://data.cms.gov/sites/default/files/2022-11/COVID-19%20Nursing%20Home%20Data%20Dictionary%2011.20.2022.pdf

#### updated data dictionary as of 7-30-23: chrome-extension://efaidnbmnnnibpcajpcglclefindmkaj/https://data.cms.gov/sites/default/files/2023-08/COVID-19%20Nursing%20Home%20Data%20Dictionary.pdf

#### guidance doc on changes to Up-to-Date vaccination definitions: chrome-extension://efaidnbmnnnibpcajpcglclefindmkaj/https://www.cdc.gov/nhsn/pdfs/hps/covidvax/UpToDateGuidance-508.pdf last updated 6-1-23


#### Resident Metrics:
* Recent Percentage of Current Residents who Received a Completed COVID-19 Vaccination at Any Time = The value of "Percentage of Current Residents who Received a Completed COVID-19 Vaccination at Any Time" for the current week, or if blank, for the prior week. Calculated as follows: <b>(Number of Residents Staying in this Facility for At Least 1 Day This Week who Received a Completed COVID-19 Vaccination at Any Time</b> / <b> Number of Residents Staying in this Facility for At Least 1 Day This Week) * 100 </b>
<br>
<br>
* <mark> Recent Percentage of Current Residents Up to Date with COVID-19 Vaccines = The value of "Percentage of Residents who are Up to Date with COVID-19 Vaccines" for the current week, or if blank, for the prior week.
Calculated as follows: <b>(Number of Residents Staying in this Facility
for At Least 1 Day This Week Up to Date with COVID-19 Vaccines</b> /
<b>Number of Residents Staying in this Facility for At Least 1 Day This
Week) * 100</b></mark>
<br>

#### Health Care Personnel Metrics:
* Recent Percentage of Current Healthcare Personnel who Received a Completed COVID-19 Vaccination at Any Time = The value of "Percentage of Current Healthcare Personnel who Received a Completed COVID-19 Vaccination at Any Time" for the current week, or if blank, for the prior week. Calculated as follows: <b> (Number of All Healthcare Personnel Eligible to Work in this Facility for At Least 1 Day This Week who Received a Completed COVID-19 Vaccination at Any Time</b> / <b>Number of All Healthcare Personnel Eligible to Work in this Facility for At Least 1
Day This Week) * 100</b>
<br>
* <mark> Recent Percentage of Current Healthcare Personnel Up to Date with COVID-19 Vaccines = The value of "Percentage of Current Healthcare Personnel who are Up to Date with COVID-19 Vaccines" for the current week, or if blank, for the prior week. Calculated as follows: <b> (Number of Healthcare Personnel Eligible to
Work in this Facility for At Least 1 Day This Week Up to Date with
COVID-19 Vaccines </b> / <b> Number of All Healthcare Personnel Eligible to
Work in this Facility for At Least 1 Day This Week) * 100 </b></mark>

?? When did they introduce the up-to-date with covid 19 vaccines metric? when did facilities start consistently reporting it?

### What does it mean to be up-to-date on COVID-19 vaccinations? To have received all vax does plus all available boosters?

### NOTE: This definition changed for Q3 2023. See guidance doc: extension://efaidnbmnnnibpcajpcglclefindmkaj/https://www.cdc.gov/nhsn/pdfs/hps/covidvax/UpToDateGuidance-508.pdf last updated 6-1-23

#### -->answer according to cdc, up-to-date means :Recipient of bivalent booster dose or completion of primary series in the past two months"  https://www.cdc.gov/nhsn/covid19/ltc-vaccination-dashboard.html. "The bivalent booster was included in the up to date definition when it became available to people 12 years of age and older on September 2, 2022."

### How do the "completed a covid-19 vaccination at any time" metrics compare to the "up-to-date with covid-19 vaccination"? 



### Read in the latest data file

In [94]:
pfile = pq.read_table("data_pre_proc/nh_all_Jun27_21_Mar19_23_concat.parquet.gzip")

# print("Column names: {}".format(pfile.column_names))
# print("Schema: {}".format(pfile.schema))

### Limit columns

In [95]:
# set keep only needed cols
needed_cols = ['week_ending',
                'federal_provider_number',
                'provider_name',
                'provider_address',
                'provider_city',
                'provider_state',
                'provider_zip_code',
                'provider_phone_number',
                'county',
                'submitted_data',
                'passed_quality_assurance_check',
                'residents_weekly_confirmed_covid_19',
                'residents_weekly_all_deaths',
                'residents_weekly_covid_19_deaths',
                'number_of_all_beds',
                'total_number_of_occupied_beds',
                'staff_weekly_confirmed_covid_19',
                'staff_weekly_covid_19_deaths',
                'shortage_of_nursing_staff',
                'shortage_of_clinical_staff',
                'shortage_of_aides',
                'weekly_resident_confirmed_covid_19_cases_per_1_000_residents',
                'number_of_staff_and_or_personnel_with_a_new_positive_covid_19_test_result',
                'Able_to_Test_or_Obtain_Resources_to_Test_All_Residents_Staff_and_or_Personnel',
                'number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week',
                'number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time',
                'recent_percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time',
                'percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time',
                'percentage_of_current_residents_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time',
                'number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week',
                'number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time',
                'percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time',
                'percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time',
                'recent_percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time',
                'Number_of_Residents_Staying_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines',
                'Recent_Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines',
                'Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines',
                'Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines',
                'Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines',
                'Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines',
                'Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines']


### Update/edit file name by date range of dataset

In [96]:
Jun27_21_Mar19_23 = pfile

nh_latest = Jun27_21_Mar19_23.to_pandas()

In [97]:
nh_latest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1380505 entries, 0 to 1380504
Columns: 121 entries, week_ending to Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines
dtypes: object(121)
memory usage: 1.2+ GB


In [98]:
nh_latest_sub = nh_latest[needed_cols]

In [99]:
nh_latest_sub.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,passed_quality_assurance_check,residents_weekly_confirmed_covid_19,residents_weekly_all_deaths,residents_weekly_covid_19_deaths,number_of_all_beds,total_number_of_occupied_beds,staff_weekly_confirmed_covid_19,staff_weekly_covid_19_deaths,shortage_of_nursing_staff,shortage_of_clinical_staff,shortage_of_aides,weekly_resident_confirmed_covid_19_cases_per_1_000_residents,number_of_staff_and_or_personnel_with_a_new_positive_covid_19_test_result,Able_to_Test_or_Obtain_Resources_to_Test_All_Residents_Staff_and_or_Personnel,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,recent_percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,recent_percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,Number_of_Residents_Staying_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines
0,2023-03-12,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,0,1,0,57,49,0,0,,,,0,0,Y,50,47,,94,94,81,70,86.42,87.5,,41,,82,24,,29.63,34.286
1,2023-03-12,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,Y,1,0,0,85,82,1,0,,,,12.2,1,Y,83,77,,92.771,92.771,181,148,81.768,82.682,,51,,61.446,1,,0.552,0.676
2,2023-03-12,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,Y,0,0,0,50,36,0,0,,,,0,0,Y,39,39,,100,100,91,75,82.418,82.418,,36,,92.308,20,,21.978,26.667
3,2023-03-12,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,Y,0,0,0,90,75,0,0,,,,0,0,Y,75,71,,94.667,94.667,98,94,95.918,95.918,,71,,94.667,30,,30.612,31.915
4,2023-03-12,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,Y,0,0,0,103,87,0,0,,,,0,0,Y,88,82,,93.182,93.182,144,119,82.639,82.639,,59,,67.045,8,,5.556,6.723


In [100]:
nh_latest_sub.tail()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,passed_quality_assurance_check,residents_weekly_confirmed_covid_19,residents_weekly_all_deaths,residents_weekly_covid_19_deaths,number_of_all_beds,total_number_of_occupied_beds,staff_weekly_confirmed_covid_19,staff_weekly_covid_19_deaths,shortage_of_nursing_staff,shortage_of_clinical_staff,shortage_of_aides,weekly_resident_confirmed_covid_19_cases_per_1_000_residents,number_of_staff_and_or_personnel_with_a_new_positive_covid_19_test_result,Able_to_Test_or_Obtain_Resources_to_Test_All_Residents_Staff_and_or_Personnel,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,recent_percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,recent_percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,Number_of_Residents_Staying_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines
1380500,2022-11-27,745003,NEURORESTORATIVE TEXAS,2301 W. CAMPBELL RD.,GARLAND,TX,75044,5124382630,Dallas,Y,N,0,0,0,12,2,0,0,,,,0,0,Y,2,0,,0,0,12,10,83.333,83.333,,0,,0,1,,8.333,10
1380501,2022-11-27,745004,THE BRIXTON AT HORSESHOE BAY,15101 WEST FM 2147,HORSESHOE BAY,TX,78657,7135531321,Burnet,Y,Y,0,0,0,120,48,0,0,,,,0,0,Y,48,45,,93.75,93.75,76,56,73.684,73.684,,27,,56.25,21,,27.632,37.5
1380502,2022-11-27,745005,CENTER AT ZARAGOZA,12660 PEBBLE HILLS BLVD.,EL PASO,TX,79938,9159901700,El Paso,Y,Y,0,0,0,80,26,0,0,,,,0,0,Y,36,29,,80.556,80.556,113,109,96.46,96.46,,1,,2.778,0,,0,0
1380503,2022-11-27,745006,FIVE POINTS NURSING AND REHABILITATION,1901 N. HAMPTON RD.,DESOTO,TX,75115,9726949810,Dallas,Y,Y,0,0,0,120,81,0,0,,,,0,0,Y,24,19,,79.167,79.167,87,57,65.517,66.279,,0,,0,0,,0,0
1380504,2022-11-27,745017,THE LODGE OF SAGINAW HEALTH AND WELLNESS,848 W MCLEROY BLVD,SAGINAW,TX,76179,6859001210,Tarrant,Y,Y,0,0,0,130,40,0,0,,,,0,0,Y,41,33,,80.488,80.488,74,69,93.243,95.833,,3,,7.317,43,,58.108,62.319


In [101]:
list(nh_latest_sub.columns)

['week_ending',
 'federal_provider_number',
 'provider_name',
 'provider_address',
 'provider_city',
 'provider_state',
 'provider_zip_code',
 'provider_phone_number',
 'county',
 'submitted_data',
 'passed_quality_assurance_check',
 'residents_weekly_confirmed_covid_19',
 'residents_weekly_all_deaths',
 'residents_weekly_covid_19_deaths',
 'number_of_all_beds',
 'total_number_of_occupied_beds',
 'staff_weekly_confirmed_covid_19',
 'staff_weekly_covid_19_deaths',
 'shortage_of_nursing_staff',
 'shortage_of_clinical_staff',
 'shortage_of_aides',
 'weekly_resident_confirmed_covid_19_cases_per_1_000_residents',
 'number_of_staff_and_or_personnel_with_a_new_positive_covid_19_test_result',
 'Able_to_Test_or_Obtain_Resources_to_Test_All_Residents_Staff_and_or_Personnel',
 'number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week',
 'number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time',
 'recent_p

In [102]:
#setting index on week ending col for quicker sorting
sorted_nh_latest_sub = nh_latest_sub.set_index('week_ending').sort_index()

sorted_nh_latest_sub.head()


,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,passed_quality_assurance_check,residents_weekly_confirmed_covid_19,residents_weekly_all_deaths,residents_weekly_covid_19_deaths,number_of_all_beds,total_number_of_occupied_beds,staff_weekly_confirmed_covid_19,staff_weekly_covid_19_deaths,shortage_of_nursing_staff,shortage_of_clinical_staff,shortage_of_aides,weekly_resident_confirmed_covid_19_cases_per_1_000_residents,number_of_staff_and_or_personnel_with_a_new_positive_covid_19_test_result,Able_to_Test_or_Obtain_Resources_to_Test_All_Residents_Staff_and_or_Personnel,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,recent_percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,recent_percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,Number_of_Residents_Staying_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines
week_ending,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-06-27,395706,"GARDENS AT EAST MOUNTAIN, THE",101 EAST MOUNTAIN DRIVE,WILKES-BARRE,PA,18702,5708255892,Luzerne,Y,Y,0,1,0,120,106,0,0,N,N,N,0,0,,106,79,,74.528,74.528,108,64,59.259,59.259,,,,0,,,0,0
2021-06-27,365937,OHMAN FAMILY LIVING AT BRIAR,15950 PIERCE ST,MIDDLEFIELD,OH,44062,4406325241,Geauga,Y,Y,0,0,0,96,81,0,0,N,N,N,0,0,,80,64,,80,82.051,105,36,34.286,34.286,,,,0,,,0,0
2021-06-27,365939,EDGEWOOD MANOR OF WELLSTON,405 NORTH PARK AVENUE,WELLSTON,OH,45692,7403845611,Jackson,Y,Y,0,0,0,50,38,0,0,N,N,N,0,0,,39,26,,66.667,66.667,57,28,49.123,49.123,,,,0,,,0,0
2021-06-27,365940,AUTUMN AEGIS NURSING HOME,1130 TOWER BLVD,LORAIN,OH,44052,4402826768,Lorain,Y,Y,0,1,0,99,64,0,0,N,N,N,0,0,,66,53,,80.303,80.303,100,53,53,53,,,,0,,,0,0
2021-06-27,365943,EUCLID SUBACUTE CARE CENTER,18901 LAKE SHORE BLVD,EUCLID,OH,44119,2166928810,Cuyahoga,Y,Y,0,0,0,40,32,0,0,N,N,N,0,0,,34,19,,55.882,55.882,241,172,71.369,71.369,,,,0,,,0,0


In [103]:
sorted_nh_latest_sub.sort_values(by=['week_ending']).tail()

,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,passed_quality_assurance_check,residents_weekly_confirmed_covid_19,residents_weekly_all_deaths,residents_weekly_covid_19_deaths,number_of_all_beds,total_number_of_occupied_beds,staff_weekly_confirmed_covid_19,staff_weekly_covid_19_deaths,shortage_of_nursing_staff,shortage_of_clinical_staff,shortage_of_aides,weekly_resident_confirmed_covid_19_cases_per_1_000_residents,number_of_staff_and_or_personnel_with_a_new_positive_covid_19_test_result,Able_to_Test_or_Obtain_Resources_to_Test_All_Residents_Staff_and_or_Personnel,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,recent_percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,recent_percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,Number_of_Residents_Staying_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines
week_ending,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-03-19,745021,LINDALE SPECIALTY CARE CENTER,13905 FM 2710,LINDALE,TX,75771,9038509380,Smith,N,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-03-19,676488,CEDAR HOLLOW REHABILITATION CENTER,5011 NORTH US HWY 75,SHERMAN,TX,75090,9037712000,Grayson,Y,Y,0,0,0,142,69,0,0,,,,0,0,Y,75,61,81.333,81.333,81.333,79,60,75.949,77.922,75.949,6,8,8,3,3.797,3.797,5
2023-03-19,676487,THE CENTER AT PARMER,13800 N FM 620 RD SB,AUSTIN,TX,78717,7372366400,Williamson,Y,Y,0,0,0,80,27,0,0,,,,0,0,Y,31,24,77.419,77.419,77.419,117,105,89.744,89.744,89.744,10,32.258,32.258,75,64.103,64.103,71.429
2023-03-19,686128,ARDIE R COPAS STATE VETERANS NURSING HOME,13000 SW TRADITION PARKWAY,PORT SAINT LUCIE,FL,34987,7722416132,St. Lucie,Y,Y,0,1,0,120,25,0,0,,,,0,7,Y,25,21,84,84,84,104,59,56.731,56.731,56.731,0,0,0,2,1.923,1.923,3.39
2023-03-19,155321,"WATERS OF FORT WAYNE SKILLED NURSING FACILITY,...",5544 E STATE BLVD,FORT WAYNE,IN,46815,2607499506,Allen,Y,Y,0,0,0,77,43,0,0,,,,0,0,Y,43,39,90.698,90.698,90.698,78,48,61.538,64.865,61.538,31,72.093,72.093,3,3.846,3.846,6.25


In [104]:
# #check for dup rows
# nh_latest_sub.loc[nh_latest_sub.duplicated(),]

In [105]:
# rename long cols
sorted_nh_latest_sub = sorted_nh_latest_sub.rename({
    'provider_zip_code':'provider_zip',
    'passed_quality_assurance_check':'data_passed_qa',
    'residents_weekly_confirmed_covid_19':'res_wk_confirmed_c19',
    'residents_weekly_all_deaths': 'res_wk_all_deaths',
    'residents_weekly_covid_19_deaths':'res_wk_c19_deaths',
    'number_of_all_beds':'num_all_beds',
    'total_number_of_occupied_beds':'num_occ_beds',
    'staff_weekly_confirmed_covid_19':'staff_wk_conf_c19',
     'staff_weekly_covid_19_deaths':'staff_wk_c19_deaths',
    'shortage_of_nursing_staff':'short_nrsg_staff',
    'shortage_of_aides':'short_aides',
    'weekly_resident_confirmed_covid_19_cases_per_1_000_residents':'wk_res_confirmed_c19_per_1K',
    'number_of_staff_and_or_personnel_with_a_new_positive_covid_19_test_result':'num_staff_new_pos_c19_test',
    'Able_to_Test_or_Obtain_Resources_to_Test_All_Residents_Staff_and_or_Personnel':'can_test_all_res_staff_pers',
    'number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week':'num_res_in_facility',
    'number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time':'num_res_comp_c19_vax_anytime',
    'number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time':'num_res_decln_c19_vax_anytime',
    'recent_percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time':'rec_pct_res_comp_c19_vax_anytime',
    'percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time':'pct_res_comp_c19_vax_anytime',
    'Number_of_Residents_Staying_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines':'num_res_utd_c19_vax',
    'Recent_Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines':'rec_pct_res_utd_c19_vax',
    'Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines':'pct_res_utd_c19_vax',
    'number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week':'num_hcp',
    'number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time':'num_hcp_comp_c19_vax_anytime',
    'number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time':'num_hcp_decln_c19_vax_anytime',
    'recent_percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time': 'rec_pct_hcp_comp_c19_vax_anytime',
    'percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time': 'pct_hcp_comp_c19_vax_anytime',
    'Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines':'num_hcp_utd_c19_vax',
    'Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines':'rec_pct_hcp_utd_c19_vax',
    'Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines':'pct_hcp_utd_c19_vax',
    'Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines':'pct_hcp_comp_c19_vax_and_utd',
   }, axis=1)

In [106]:
sorted_nh_latest_sub.head()

,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip,provider_phone_number,county,submitted_data,data_passed_qa,res_wk_confirmed_c19,res_wk_all_deaths,res_wk_c19_deaths,num_all_beds,num_occ_beds,staff_wk_conf_c19,staff_wk_c19_deaths,short_nrsg_staff,shortage_of_clinical_staff,short_aides,wk_res_confirmed_c19_per_1K,num_staff_new_pos_c19_test,can_test_all_res_staff_pers,num_res_in_facility,num_res_comp_c19_vax_anytime,rec_pct_res_comp_c19_vax_anytime,pct_res_comp_c19_vax_anytime,percentage_of_current_residents_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,num_hcp,num_hcp_comp_c19_vax_anytime,pct_hcp_comp_c19_vax_anytime,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,rec_pct_hcp_comp_c19_vax_anytime,num_res_utd_c19_vax,rec_pct_res_utd_c19_vax,pct_res_utd_c19_vax,num_hcp_utd_c19_vax,rec_pct_hcp_utd_c19_vax,pct_hcp_utd_c19_vax,pct_hcp_comp_c19_vax_and_utd
week_ending,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-06-27,395706,"GARDENS AT EAST MOUNTAIN, THE",101 EAST MOUNTAIN DRIVE,WILKES-BARRE,PA,18702,5708255892,Luzerne,Y,Y,0,1,0,120,106,0,0,N,N,N,0,0,,106,79,,74.528,74.528,108,64,59.259,59.259,,,,0,,,0,0
2021-06-27,365937,OHMAN FAMILY LIVING AT BRIAR,15950 PIERCE ST,MIDDLEFIELD,OH,44062,4406325241,Geauga,Y,Y,0,0,0,96,81,0,0,N,N,N,0,0,,80,64,,80,82.051,105,36,34.286,34.286,,,,0,,,0,0
2021-06-27,365939,EDGEWOOD MANOR OF WELLSTON,405 NORTH PARK AVENUE,WELLSTON,OH,45692,7403845611,Jackson,Y,Y,0,0,0,50,38,0,0,N,N,N,0,0,,39,26,,66.667,66.667,57,28,49.123,49.123,,,,0,,,0,0
2021-06-27,365940,AUTUMN AEGIS NURSING HOME,1130 TOWER BLVD,LORAIN,OH,44052,4402826768,Lorain,Y,Y,0,1,0,99,64,0,0,N,N,N,0,0,,66,53,,80.303,80.303,100,53,53,53,,,,0,,,0,0
2021-06-27,365943,EUCLID SUBACUTE CARE CENTER,18901 LAKE SHORE BLVD,EUCLID,OH,44119,2166928810,Cuyahoga,Y,Y,0,0,0,40,32,0,0,N,N,N,0,0,,34,19,,55.882,55.882,241,172,71.369,71.369,,,,0,,,0,0


In [107]:
# resetting index to keep the date column
sorted_nh_latest_sub = sorted_nh_latest_sub.reset_index()

In [108]:
sorted_nh_latest_sub.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip,provider_phone_number,county,submitted_data,data_passed_qa,res_wk_confirmed_c19,res_wk_all_deaths,res_wk_c19_deaths,num_all_beds,num_occ_beds,staff_wk_conf_c19,staff_wk_c19_deaths,short_nrsg_staff,shortage_of_clinical_staff,short_aides,wk_res_confirmed_c19_per_1K,num_staff_new_pos_c19_test,can_test_all_res_staff_pers,num_res_in_facility,num_res_comp_c19_vax_anytime,rec_pct_res_comp_c19_vax_anytime,pct_res_comp_c19_vax_anytime,percentage_of_current_residents_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,num_hcp,num_hcp_comp_c19_vax_anytime,pct_hcp_comp_c19_vax_anytime,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,rec_pct_hcp_comp_c19_vax_anytime,num_res_utd_c19_vax,rec_pct_res_utd_c19_vax,pct_res_utd_c19_vax,num_hcp_utd_c19_vax,rec_pct_hcp_utd_c19_vax,pct_hcp_utd_c19_vax,pct_hcp_comp_c19_vax_and_utd
0,2021-06-27,395706,"GARDENS AT EAST MOUNTAIN, THE",101 EAST MOUNTAIN DRIVE,WILKES-BARRE,PA,18702,5708255892,Luzerne,Y,Y,0,1,0,120,106,0,0,N,N,N,0,0,,106,79,,74.528,74.528,108,64,59.259,59.259,,,,0,,,0,0
1,2021-06-27,365937,OHMAN FAMILY LIVING AT BRIAR,15950 PIERCE ST,MIDDLEFIELD,OH,44062,4406325241,Geauga,Y,Y,0,0,0,96,81,0,0,N,N,N,0,0,,80,64,,80,82.051,105,36,34.286,34.286,,,,0,,,0,0
2,2021-06-27,365939,EDGEWOOD MANOR OF WELLSTON,405 NORTH PARK AVENUE,WELLSTON,OH,45692,7403845611,Jackson,Y,Y,0,0,0,50,38,0,0,N,N,N,0,0,,39,26,,66.667,66.667,57,28,49.123,49.123,,,,0,,,0,0
3,2021-06-27,365940,AUTUMN AEGIS NURSING HOME,1130 TOWER BLVD,LORAIN,OH,44052,4402826768,Lorain,Y,Y,0,1,0,99,64,0,0,N,N,N,0,0,,66,53,,80.303,80.303,100,53,53,53,,,,0,,,0,0
4,2021-06-27,365943,EUCLID SUBACUTE CARE CENTER,18901 LAKE SHORE BLVD,EUCLID,OH,44119,2166928810,Cuyahoga,Y,Y,0,0,0,40,32,0,0,N,N,N,0,0,,34,19,,55.882,55.882,241,172,71.369,71.369,,,,0,,,0,0


In [109]:
# subset cols
nh_latest_sub_2 = sorted_nh_latest_sub[[
                 'week_ending',
                 'federal_provider_number',
                 'provider_name',
                 'provider_address',
                 'provider_city',
                 'provider_state',
                 'provider_zip',
                 'provider_phone_number',
                 'county',
                 'submitted_data',
                 'data_passed_qa',
                 'res_wk_confirmed_c19',
                 'res_wk_all_deaths',
                 'res_wk_c19_deaths',
                 'num_all_beds',
                 'num_occ_beds',
                 'staff_wk_conf_c19',
                 'staff_wk_c19_deaths',
                 'short_nrsg_staff',
                 'short_aides',
                 'wk_res_confirmed_c19_per_1K',
                 'num_staff_new_pos_c19_test',
                 'can_test_all_res_staff_pers',
                 'num_res_in_facility',
                 'num_res_comp_c19_vax_anytime',
                 'rec_pct_res_comp_c19_vax_anytime',
                 'pct_res_comp_c19_vax_anytime',
                 'num_res_utd_c19_vax',
                 'rec_pct_res_utd_c19_vax',
                 'num_hcp',
                 'num_hcp_comp_c19_vax_anytime',
                 'rec_pct_hcp_comp_c19_vax_anytime',
                 'pct_hcp_comp_c19_vax_anytime',
                 'num_hcp_utd_c19_vax',
                 'rec_pct_hcp_utd_c19_vax',
                 'pct_hcp_utd_c19_vax',
                 'pct_hcp_comp_c19_vax_and_utd'
]]

nh_latest_sub_2.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip,provider_phone_number,county,submitted_data,data_passed_qa,res_wk_confirmed_c19,res_wk_all_deaths,res_wk_c19_deaths,num_all_beds,num_occ_beds,staff_wk_conf_c19,staff_wk_c19_deaths,short_nrsg_staff,short_aides,wk_res_confirmed_c19_per_1K,num_staff_new_pos_c19_test,can_test_all_res_staff_pers,num_res_in_facility,num_res_comp_c19_vax_anytime,rec_pct_res_comp_c19_vax_anytime,pct_res_comp_c19_vax_anytime,num_res_utd_c19_vax,rec_pct_res_utd_c19_vax,num_hcp,num_hcp_comp_c19_vax_anytime,rec_pct_hcp_comp_c19_vax_anytime,pct_hcp_comp_c19_vax_anytime,num_hcp_utd_c19_vax,rec_pct_hcp_utd_c19_vax,pct_hcp_utd_c19_vax,pct_hcp_comp_c19_vax_and_utd
0,2021-06-27,395706,"GARDENS AT EAST MOUNTAIN, THE",101 EAST MOUNTAIN DRIVE,WILKES-BARRE,PA,18702,5708255892,Luzerne,Y,Y,0,1,0,120,106,0,0,N,N,0,0,,106,79,,74.528,,,108,64,,59.259,,,0,0
1,2021-06-27,365937,OHMAN FAMILY LIVING AT BRIAR,15950 PIERCE ST,MIDDLEFIELD,OH,44062,4406325241,Geauga,Y,Y,0,0,0,96,81,0,0,N,N,0,0,,80,64,,80,,,105,36,,34.286,,,0,0
2,2021-06-27,365939,EDGEWOOD MANOR OF WELLSTON,405 NORTH PARK AVENUE,WELLSTON,OH,45692,7403845611,Jackson,Y,Y,0,0,0,50,38,0,0,N,N,0,0,,39,26,,66.667,,,57,28,,49.123,,,0,0
3,2021-06-27,365940,AUTUMN AEGIS NURSING HOME,1130 TOWER BLVD,LORAIN,OH,44052,4402826768,Lorain,Y,Y,0,1,0,99,64,0,0,N,N,0,0,,66,53,,80.303,,,100,53,,53,,,0,0
4,2021-06-27,365943,EUCLID SUBACUTE CARE CENTER,18901 LAKE SHORE BLVD,EUCLID,OH,44119,2166928810,Cuyahoga,Y,Y,0,0,0,40,32,0,0,N,N,0,0,,34,19,,55.882,,,241,172,,71.369,,,0,0


In [110]:
#subset to our date range: Week ending 7-4-2021 through week ending 7-3-2022

#define variables for date range
start_date = '2021-07-04'
end_date = '2022-07-03'

#convert week_ending to datetime
nh_latest_sub_2['week_ending'] = pd.to_datetime(nh_latest_sub_2['week_ending'])

# create week start date col (should be a Mon)
nh_latest_sub_2['week_start'] = nh_latest_sub_2['week_ending'] - pd.Timedelta(days = 6)

nh_latest_sub_2['month_year'] = pd.to_datetime(nh_latest_sub_2['week_start']).dt.to_period('M')

# converting period to string for ease of manipulation with polars
nh_latest_sub_2['month_year'] = nh_latest_sub_2['month_year'].dt.strftime('%Y-%m')

#create a boolean mask for rows within the date range
dr_mask = (nh_latest_sub_2['week_ending'] >= start_date) & (nh_latest_sub_2['week_ending'] <= end_date)

# Apply the boolean mask to subset the DataFrame
dr_nh_latest_sub_2 = nh_latest_sub_2[dr_mask]


In [111]:
dr_nh_latest_sub_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 806019 entries, 15257 to 821275
Data columns (total 39 columns):
 #   Column                            Non-Null Count   Dtype         
---  ------                            --------------   -----         
 0   week_ending                       806019 non-null  datetime64[ns]
 1   federal_provider_number           806019 non-null  object        
 2   provider_name                     806019 non-null  object        
 3   provider_address                  806019 non-null  object        
 4   provider_city                     806019 non-null  object        
 5   provider_state                    806019 non-null  object        
 6   provider_zip                      806019 non-null  object        
 7   provider_phone_number             806019 non-null  object        
 8   county                            806019 non-null  object        
 9   submitted_data                    806019 non-null  object        
 10  data_passed_qa              

In [112]:
#replace spaces with null
dr_nh_latest_sub_2.replace(['',' ', '\t'], np.nan, inplace=True)
dr_nh_latest_sub_2.head(10)

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip,provider_phone_number,county,submitted_data,data_passed_qa,res_wk_confirmed_c19,res_wk_all_deaths,res_wk_c19_deaths,num_all_beds,num_occ_beds,staff_wk_conf_c19,staff_wk_c19_deaths,short_nrsg_staff,short_aides,wk_res_confirmed_c19_per_1K,num_staff_new_pos_c19_test,can_test_all_res_staff_pers,num_res_in_facility,num_res_comp_c19_vax_anytime,rec_pct_res_comp_c19_vax_anytime,pct_res_comp_c19_vax_anytime,num_res_utd_c19_vax,rec_pct_res_utd_c19_vax,num_hcp,num_hcp_comp_c19_vax_anytime,rec_pct_hcp_comp_c19_vax_anytime,pct_hcp_comp_c19_vax_anytime,num_hcp_utd_c19_vax,rec_pct_hcp_utd_c19_vax,pct_hcp_utd_c19_vax,pct_hcp_comp_c19_vax_and_utd,week_start,month_year
15257,2021-07-04,365961,PIKETON NURSING CENTER,300 OVERLOOK DRIVE,PIKETON,OH,45661,7402894074,Pike,Y,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-06-28,2021-06
15258,2021-07-04,365956,KENDAL AT OBERLIN,600 KENDAL DRIVE,OBERLIN,OH,44074,4407750094,Lorain,Y,N,0,0,0,12,0,0,0,N,N,NaN,0,NaN,0,0,NaN,0,NaN,NaN,141,112,NaN,79.433,NaN,NaN,0,0,2021-06-28,2021-06
15259,2021-07-04,365953,OTTERBEIN ST MARYS RETIREMENT COMMUNITY,11230 STATE ROUTE 364,ST MARYS,OH,45885,4193946330,Auglaize,Y,Y,0,0,0,76,49,0,0,N,N,0,0,NaN,52,44,NaN,84.615,NaN,NaN,190,60,NaN,31.579,NaN,NaN,0,0,2021-06-28,2021-06
15260,2021-07-04,365952,RIDGEWOOD MANOR,3231 MANLEY ROAD,MAUMEE,OH,43537,4198651248,Lucas,Y,Y,0,2,0,90,53,0,0,N,N,0,0,NaN,56,38,NaN,67.857,NaN,NaN,80,32,NaN,40,NaN,NaN,0,0,2021-06-28,2021-06
15261,2021-07-04,365945,CRYSTAL CARE CENTER OF MANSFIE,1159 WYANDOTTE AVE,MANSFIELD,OH,44906,4197472666,Richland,Y,Y,0,0,0,74,57,0,0,N,N,0,0,NaN,61,54,NaN,88.525,NaN,NaN,98,35,NaN,35.714,NaN,NaN,0,0,2021-06-28,2021-06
15262,2021-07-04,365949,CONCORD CARE CENTER OF CORTLAND,4250 SODOM HUTCHINGS RD,CORTLAND,OH,44410,3306377906,Trumbull,Y,Y,0,0,0,48,26,0,0,N,N,0,0,NaN,26,21,NaN,80.769,NaN,NaN,21,11,NaN,52.381,NaN,NaN,0,0,2021-06-28,2021-06
15263,2021-07-04,365947,OHMAN FAMILY LIVING AT HOLLY,10190 FAIRMOUNT RD,NEWBURY,OH,44065,4403388220,Geauga,Y,Y,0,0,0,92,77,1,0,Y,Y,0,1,NaN,86,77,NaN,89.535,NaN,NaN,139,92,NaN,66.187,NaN,NaN,0,0,2021-06-28,2021-06
15264,2021-07-04,365946,DIVERSICARE OF ST THERESA,7010 ROWAN HILL DRIVE,CINCINNATI,OH,45227,5132717010,Hamilton,Y,Y,0,0,0,191,65,0,0,N,N,0,0,NaN,68,50,NaN,73.529,NaN,NaN,74,23,NaN,31.081,NaN,NaN,0,0,2021-06-28,2021-06
15265,2021-07-04,365981,GEM CITY HEALTHCARE AND REHABILITATION CENTER,323 FOREST AVENUE,DAYTON,OH,45405,9372240793,Montgomery,Y,Y,0,0,0,78,47,0,0,N,N,0,0,NaN,47,40,NaN,85.106,NaN,NaN,45,13,NaN,28.889,NaN,NaN,0,0,2021-06-28,2021-06
15266,2021-07-04,365943,EUCLID SUBACUTE CARE CENTER,18901 LAKE SHORE BLVD,EUCLID,OH,44119,2166928810,Cuyahoga,Y,Y,0,1,0,40,32,0,0,N,N,0,0,NaN,42,27,NaN,64.286,NaN,NaN,232,179,NaN,77.155,NaN,NaN,0,0,2021-06-28,2021-06


In [113]:
#revise nums to float cols
float_cols = [
    'res_wk_confirmed_c19',
    'res_wk_all_deaths',
     'res_wk_c19_deaths',
     'num_all_beds',
     'num_occ_beds',
     'staff_wk_conf_c19',
     'staff_wk_c19_deaths',
     'wk_res_confirmed_c19_per_1K',
     'num_staff_new_pos_c19_test',
     'num_res_in_facility',
     'num_res_comp_c19_vax_anytime',
     'rec_pct_res_comp_c19_vax_anytime',
     'pct_res_comp_c19_vax_anytime',
     'num_res_utd_c19_vax',
     'rec_pct_res_utd_c19_vax',
     'num_hcp',
     'num_hcp_comp_c19_vax_anytime',
     'rec_pct_hcp_comp_c19_vax_anytime',
     'pct_hcp_comp_c19_vax_anytime',
     'num_hcp_utd_c19_vax',
     'rec_pct_hcp_utd_c19_vax',
     'pct_hcp_utd_c19_vax',
     'pct_hcp_comp_c19_vax_and_utd'
]

dr_nh_latest_sub_2[float_cols]=dr_nh_latest_sub_2[float_cols].astype('float')


In [114]:
# rounding metrics we want and expressing them as pct

#anytime vax metrics
dr_nh_latest_sub_2['rev_pct_res_anytime_c19_vax'] = 100*(dr_nh_latest_sub_2.num_res_comp_c19_vax_anytime/dr_nh_latest_sub_2.num_res_in_facility).round(2)
dr_nh_latest_sub_2['rev_pct_hcp_anytime_c19_vax'] = 100*(dr_nh_latest_sub_2.num_hcp_comp_c19_vax_anytime/dr_nh_latest_sub_2.num_hcp).round(2)
dr_nh_latest_sub_2['rev_ratio_res_anytime_c19_vax'] = 10*(dr_nh_latest_sub_2.num_res_comp_c19_vax_anytime/dr_nh_latest_sub_2.num_res_in_facility).round(2)
dr_nh_latest_sub_2['rev_ratio_hcp_anytime_c19_vax'] = 10*(dr_nh_latest_sub_2.num_hcp_comp_c19_vax_anytime/dr_nh_latest_sub_2.num_hcp).round(2)

#up to date vax metrics
dr_nh_latest_sub_2['rev_pct_res_utd_c19_vax'] = 100*(dr_nh_latest_sub_2.num_res_utd_c19_vax/dr_nh_latest_sub_2.num_res_in_facility).round(2)
dr_nh_latest_sub_2['rev_pct_hcp_utd_c19_vax'] = 100*(dr_nh_latest_sub_2.num_hcp_utd_c19_vax/dr_nh_latest_sub_2.num_hcp).round(2)

# cases
dr_nh_latest_sub_2['rev_pct_res_wk_c19_deaths'] = 100 *(dr_nh_latest_sub_2['res_wk_c19_deaths']/dr_nh_latest_sub_2['num_res_in_facility']).round(2)
dr_nh_latest_sub_2['rev_pct_staff_wk_c19_deaths'] = 100 *(dr_nh_latest_sub_2['staff_wk_c19_deaths']/dr_nh_latest_sub_2['num_hcp']).round(2)

# deaths
dr_nh_latest_sub_2['rev_pct_res_wk_confirmed_c19'] = 100 *(dr_nh_latest_sub_2['res_wk_confirmed_c19']/dr_nh_latest_sub_2['num_res_in_facility']).round(2)
dr_nh_latest_sub_2['rev_pct_staff_wk_conf_c19'] = 100 *(dr_nh_latest_sub_2['staff_wk_conf_c19']/dr_nh_latest_sub_2['num_hcp']).round(2)


dr_nh_latest_sub_2.head()


,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip,provider_phone_number,county,submitted_data,data_passed_qa,res_wk_confirmed_c19,res_wk_all_deaths,res_wk_c19_deaths,num_all_beds,num_occ_beds,staff_wk_conf_c19,staff_wk_c19_deaths,short_nrsg_staff,short_aides,wk_res_confirmed_c19_per_1K,num_staff_new_pos_c19_test,can_test_all_res_staff_pers,num_res_in_facility,num_res_comp_c19_vax_anytime,rec_pct_res_comp_c19_vax_anytime,pct_res_comp_c19_vax_anytime,num_res_utd_c19_vax,rec_pct_res_utd_c19_vax,num_hcp,num_hcp_comp_c19_vax_anytime,rec_pct_hcp_comp_c19_vax_anytime,pct_hcp_comp_c19_vax_anytime,num_hcp_utd_c19_vax,rec_pct_hcp_utd_c19_vax,pct_hcp_utd_c19_vax,pct_hcp_comp_c19_vax_and_utd,week_start,month_year,rev_pct_res_anytime_c19_vax,rev_pct_hcp_anytime_c19_vax,rev_ratio_res_anytime_c19_vax,rev_ratio_hcp_anytime_c19_vax,rev_pct_res_utd_c19_vax,rev_pct_hcp_utd_c19_vax,rev_pct_res_wk_c19_deaths,rev_pct_staff_wk_c19_deaths,rev_pct_res_wk_confirmed_c19,rev_pct_staff_wk_conf_c19
15257,2021-07-04,365961,PIKETON NURSING CENTER,300 OVERLOOK DRIVE,PIKETON,OH,45661,7402894074,Pike,Y,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-06-28,2021-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15258,2021-07-04,365956,KENDAL AT OBERLIN,600 KENDAL DRIVE,OBERLIN,OH,44074,4407750094,Lorain,Y,N,0.0,0.0,0.0,12.0,0.0,0.0,0.0,N,N,NaN,0.0,NaN,0.0,0.0,NaN,0.000,NaN,NaN,141.0,112.0,NaN,79.433,NaN,NaN,0.0,0.0,2021-06-28,2021-06,NaN,79.0,NaN,7.9,NaN,NaN,NaN,0.0,NaN,0.0
15259,2021-07-04,365953,OTTERBEIN ST MARYS RETIREMENT COMMUNITY,11230 STATE ROUTE 364,ST MARYS,OH,45885,4193946330,Auglaize,Y,Y,0.0,0.0,0.0,76.0,49.0,0.0,0.0,N,N,0.0,0.0,NaN,52.0,44.0,NaN,84.615,NaN,NaN,190.0,60.0,NaN,31.579,NaN,NaN,0.0,0.0,2021-06-28,2021-06,85.0,32.0,8.5,3.2,NaN,NaN,0.0,0.0,0.0,0.0
15260,2021-07-04,365952,RIDGEWOOD MANOR,3231 MANLEY ROAD,MAUMEE,OH,43537,4198651248,Lucas,Y,Y,0.0,2.0,0.0,90.0,53.0,0.0,0.0,N,N,0.0,0.0,NaN,56.0,38.0,NaN,67.857,NaN,NaN,80.0,32.0,NaN,40.000,NaN,NaN,0.0,0.0,2021-06-28,2021-06,68.0,40.0,6.8,4.0,NaN,NaN,0.0,0.0,0.0,0.0
15261,2021-07-04,365945,CRYSTAL CARE CENTER OF MANSFIE,1159 WYANDOTTE AVE,MANSFIELD,OH,44906,4197472666,Richland,Y,Y,0.0,0.0,0.0,74.0,57.0,0.0,0.0,N,N,0.0,0.0,NaN,61.0,54.0,NaN,88.525,NaN,NaN,98.0,35.0,NaN,35.714,NaN,NaN,0.0,0.0,2021-06-28,2021-06,89.0,36.0,8.9,3.6,NaN,NaN,0.0,0.0,0.0,0.0


In [115]:
dr_nh_latest_sub_2.provider_name = dr_nh_latest_sub_2.provider_name.str.replace(',','')
dr_nh_latest_sub_2.provider_name = dr_nh_latest_sub_2.provider_name.str.replace('"','')

## Create polars df for group bys

In [122]:
# create a polars df
pl_dr_nh_latest_sub_2 = pl.DataFrame(dr_nh_latest_sub_2)

### Counting NaN values by category

In [123]:
# Define columns to count NaNs for
columns_to_count = ['num_res_in_facility', 'num_res_comp_c19_vax_anytime', 'num_res_utd_c19_vax', 'num_hcp', 'num_hcp_comp_c19_vax_anytime', 'num_hcp_utd_c19_vax']

In [125]:
# Group by state, city, facility, and date columns
grouped = pl_dr_nh_latest_sub_2.groupby(['month_year','provider_state', 'county','federal_provider_number','submitted_data', 'data_passed_qa'])

# # Define function to count NaN values
# def count_nans(series):
#     return series.is_null().sum()

# function to count NaN values for the specified columns within each group
nan_counts = grouped.agg(pl.col(columns_to_count).is_null().sum())

# Convert the resulting Polars DataFrame back to pandas for easier printing
grp_nan_counts = nan_counts.to_pandas().sort_values(
    by=['provider_state', 'county','federal_provider_number','month_year','num_res_in_facility','num_hcp','submitted_data', 'data_passed_qa'],
                                                       # ascending = False
                                                      )

In [126]:
grp_nan_counts.head()

,month_year,provider_state,county,federal_provider_number,submitted_data,data_passed_qa,num_res_in_facility,num_res_comp_c19_vax_anytime,num_res_utd_c19_vax,num_hcp,num_hcp_comp_c19_vax_anytime,num_hcp_utd_c19_vax
102829,2021-06,AK,Anchorage,025018,Y,Y,0,0,1,0,0,1
56399,2021-07,AK,Anchorage,025018,Y,Y,0,0,4,0,0,4
60279,2021-08,AK,Anchorage,025018,Y,Y,0,0,5,0,0,5
164103,2021-09,AK,Anchorage,025018,Y,Y,0,0,4,0,0,4
68080,2021-10,AK,Anchorage,025018,Y,Y,0,0,4,0,0,4


### What is the percent of pass qa = N for each facility, for each month?

In [127]:
#By State?
state_qa=grp_nan_counts.groupby(['provider_state','county','federal_provider_number','month_year'], dropna=False)['data_passed_qa'].value_counts(normalize=True).to_frame(name='pct').reset_index()

state_qa.head()

,provider_state,county,federal_provider_number,month_year,data_passed_qa,pct
0,AK,Anchorage,025018,2021-06,Y,1.0
1,AK,Anchorage,025018,2021-07,Y,1.0
2,AK,Anchorage,025018,2021-08,Y,1.0
3,AK,Anchorage,025018,2021-09,Y,1.0
4,AK,Anchorage,025018,2021-10,Y,1.0


In [88]:
#Sorting by not passed qa descending
state_qa.loc[state_qa['data_passed_qa']=='N'].sort_values(by=['pct'], ascending=False)

,provider_state,provider_city,federal_provider_number,month_year,data_passed_qa,pct
90298,MI,LIVONIA,235604,2021-06,N,1.0
77097,LA,NEW ORLEANS,195631,2022-01,N,1.0
77095,LA,NEW ORLEANS,195631,2021-11,N,1.0
77094,LA,NEW ORLEANS,195631,2021-10,N,1.0
77075,LA,NEW ORLEANS,195614,2021-06,N,1.0
77055,LA,NEW ORLEANS,195517,2022-01,N,1.0
77054,LA,NEW ORLEANS,195517,2021-12,N,1.0
77053,LA,NEW ORLEANS,195517,2021-11,N,1.0
77052,LA,NEW ORLEANS,195517,2021-10,N,1.0
77051,LA,NEW ORLEANS,195517,2021-09,N,1.0


## Look at NaN count in Jefferson County, KY

In [80]:

nan_counts_result.loc[(nan_counts_result['provider_state']=='KY') & (nan_counts_result['county]=='Jefferson')]


,month_year,provider_state,provider_city,federal_provider_number,submitted_data,data_passed_qa,num_res_in_facility,num_res_comp_c19_vax_anytime,num_res_utd_c19_vax,num_hcp,num_hcp_comp_c19_vax_anytime,num_hcp_utd_c19_vax
153630,2021-06,KY,LOUISVILLE,185029,Y,Y,0,0,1,0,0,1
107534,2021-07,KY,LOUISVILLE,185029,Y,Y,0,0,4,0,0,4
61291,2021-08,KY,LOUISVILLE,185029,Y,Y,0,0,5,0,0,5
115292,2021-09,KY,LOUISVILLE,185029,Y,Y,0,0,4,0,0,4
19005,2021-10,KY,LOUISVILLE,185029,Y,Y,0,0,4,0,0,4
173008,2021-11,KY,LOUISVILLE,185029,Y,Y,0,0,5,0,0,5
76735,2021-12,KY,LOUISVILLE,185029,Y,Y,0,0,4,0,0,4
30695,2022-01,KY,LOUISVILLE,185029,Y,Y,0,0,5,0,0,5
134326,2022-02,KY,LOUISVILLE,185029,Y,Y,0,0,4,0,0,4
188326,2022-03,KY,LOUISVILLE,185029,Y,Y,0,0,4,0,0,4


In [ ]:
df_2021_06_27_2022_01_02 = dr_nh_latest_sub_2.loc[(dr_nh_latest_sub_2['week_ending'] >= '2021-06-27') & (dr_nh_latest_sub_2['week_ending'] <= '2022-01-02')]

## 8-28 Left off: Next steps: 
### read in new data.
### create facility-level datasets =
#### df_2021_06_27_2022_01_02,
#### df_2022_01_09_2022_07_03,
#### df_2022_07_10_2023_01_01
#### df_2023_01_08_2023_08_06
### groupby and agg to city level
### groupby and agg to state level
### export data to post proc folder

In [ ]:
#subset to only records where data_passed_qa = Y
nh_latest_sub_2_y=nh_latest_sub_2[nh_latest_sub_2['data_passed_qa']=='Y']
nh_latest_sub_2_y.reset_index(drop=True)
nh_latest_sub_2_y.head()

In [ ]:
nh_latest_sub_2_y=nh_latest_sub_2_y.loc[(nh_latest_sub_2_y['week_ending'] >= '2021-06-27') & (nh_latest_sub_2_y['week_ending'] <= '2022-01-02')]

In [ ]:
nh_latest_sub_2_y.provider_name.dtypes

In [ ]:
nh_latest_sub_2_y.data_passed_qa.value_counts()

In [ ]:
nh_latest_sub_2_y.submitted_data.value_counts()

In [ ]:
nh_latest_sub_2_y=nh_latest_sub_2_y.sort_values(by='week_start')

In [ ]:
nh_latest_sub_2_y.tail()

In [ ]:
nh_latest_sub_2_y.head()

### Export files to data post proc folder

In [ ]:
nh_latest_sub_2_y.to_csv("data/data_post_proc/nh_latest_sub_2_y.csv", encoding='utf-8', sep = '|',index=False)
#nh_latest_sub_2_y.to_parquet("data/data_post_proc/nh_latest_sub_2_y.parquet", engine='auto', compression='snappy', index=None, partition_cols=None)

## Below is analysis at city/county/state level

### Group to county level

In [ ]:
nh_latest_sub_end_22_county_grp=nh_latest_sub_end_22.groupby(['week_start','provider_state','county','data_passed_qa'],dropna=False)['num_res_in_facility','num_res_utd_c19_vax','num_hcp','num_hcp_utd_c19_vax'].sum().reset_index()
nh_latest_sub_end_22_county_grp.head()

In [ ]:
nh_latest_sub_end_22_county_grp.loc[nh_latest_sub_end_22_county_grp['data_passed_qa']=='Y'].num_res_utd_c19_vax.sum()


In [ ]:
# create rate cols
nh_latest_sub_end_22_county_grp['pct_res_utd_c19_vax']= (nh_latest_sub_end_22_county_grp['num_res_utd_c19_vax']/nh_latest_sub_end_22_county_grp['num_res_in_facility']).round(2)
nh_latest_sub_end_22_county_grp['pct_hcp_utd_c19_vax'] = (nh_latest_sub_end_22_county_grp['num_hcp_utd_c19_vax']/nh_latest_sub_end_22_county_grp['num_hcp']).round(2)
nh_latest_sub_end_22_county_grp.head()


In [ ]:
nh_latest_sub_end_22_county_grp_y=nh_latest_sub_end_22_county_grp[nh_latest_sub_end_22_county_grp.data_passed_qa == 'Y'].sort_values(by='week_start').reset_index()
nh_latest_sub_end_22_county_grp_y.head()



### Group to state level

In [ ]:
nh_latest_sub_end_22_state_grp=nh_latest_sub_end_22.groupby(['week_start','provider_state','data_passed_qa'],dropna=False)['num_res_in_facility','num_res_utd_c19_vax','num_hcp','num_hcp_utd_c19_vax'].sum().reset_index()
nh_latest_sub_end_22_state_grp.head()


In [ ]:
# create rate cols
nh_latest_sub_end_22_state_grp['pct_res_utd_c19_vax']= (nh_latest_sub_end_22_state_grp['num_res_utd_c19_vax']/nh_latest_sub_end_22_state_grp['num_res_in_facility']).round(2)
nh_latest_sub_end_22_state_grp['pct_hcp_utd_c19_vax'] = (nh_latest_sub_end_22_state_grp['num_hcp_utd_c19_vax']/nh_latest_sub_end_22_state_grp['num_hcp']).round(2)
nh_latest_sub_end_22_state_grp.head()
